In [2]:
import pandas as pd
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

In [3]:
# Charger les données d'entraînement et de test
train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('Test.csv')

# Séparer les features et la cible dans les données d'entraînement
X_train = train_data.drop(columns=['Exited'])
y_train = train_data['Exited']

In [ ]:
# Prétraitement des données
numeric_features = X_train.select_dtypes(include=['int64', 'float64']).columns
categorical_features = X_train.select_dtypes(include=['object']).columns

In [ ]:
# Créer un preprocessor
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_features),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
    ])

# Modèle
model = LogisticRegression(max_iter=1000)

# Pipeline
pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                           ('classifier', model)])


In [ ]:
# Définition de la grille des hyperparamètres à rechercher
param_grid = {
    'classifier__C': [0.001, 0.01, 0.1, 1, 10, 100],
    'classifier__penalty': ['l1', 'l2']
}

In [ ]:
# Recherche par grille avec validation croisée
grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='roc_auc')
grid_search.fit(X_train, y_train)

# Meilleurs hyperparamètres trouvés
best_params = grid_search.best_params_
print("Meilleurs hyperparamètres:", best_params)

In [ ]:
# Utilisation du meilleur modèle trouvé
best_model = grid_search.best_estimator_

In [ ]:
# Entraîner le meilleur modèle sur toutes les données d'entraînement
best_model.fit(X_train, y_train)

# Prétraitement des données de test
X_test = test_data

# Prédire la probabilité de 'Exited' pour les données de test
test_probabilities = best_model.predict_proba(X_test)[:, 1]

In [1]:

# Créer un DataFrame pour les probabilités prédites
submission_df = pd.DataFrame({'id': test_data['id'], 'Exited': test_probabilities})

# Enregistrer les prédictions au format CSV
submission_df.to_csv('prediction.csv', index=False)

c:\Users\MEGHANE\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\model_selection\_validation.py:547: FitFailedWarning: 
30 fits failed out of a total of 60.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
30 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\MEGHANE\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\model_selection\_validation.py", line 895, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\MEGHANE\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py", line 1474, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\MEGHAN

Meilleurs hyperparamètres: {'classifier__C': 0.1, 'classifier__penalty': 'l2'}
